<center>
<h1>CSC-343 Artificial Intelligence</h1>
<h1>Programming Assignment 6.1.</h1>

<br/>
<hr/>
<br/>

<img width="50%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.5./infinite_monkey.png">

<br/>
<hr/>
<br/>

</center>

In [1]:
import pandas as pd

corpus = ["To be or not to be ", \
          "To be a student is hard ", \
          "Be there or be square ! ", \
          "Be kind . Dont be mean"]

def tokenize(text):
  corp = ""
  for string in corpus:
    corp += string

  corp = corp.lower()
  all_tokens = corp.split(" ")
  vocab = set(all_tokens)
  return all_tokens, vocab

all_tokens, vocab = tokenize(corpus)
tmpList = [0, 1, 2, 3, 4, 5]

df = pd.DataFrame(0, index = vocab, columns = tmpList)
df = df.sort_index(0)

<ipython-input-1-17227752eb3a>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  df = df.sort_index(0)


<br>
<hr>
<br>

The corpus above has four sentences, each comprised of six tokens. 

Treat the corpus above as a Bayesian network (or Markov Chain) with 6 variables: one for each location.

The four sentences therefore are four events, in the probabilistic sense. 

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.5./markov_chain.png">


<br>
<hr>
<br>


# Step 1. Compute Location Counts

<img width="25%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.5./loc_counts.png">


In [2]:
for index, word in enumerate(all_tokens):
  ind = str(index % 6)
  df.loc[word, index %6] = df.loc[word, index %6] + 1

df


,0,1,2,3,4,5
!,0,0,0,0,0,1
.,0,0,1,0,0,0
a,0,0,1,0,0,0
be,2,2,0,1,1,1
dont,0,0,0,1,0,0
hard,0,0,0,0,0,1
is,0,0,0,0,1,0
kind,0,1,0,0,0,0
mean,0,0,0,0,0,1
not,0,0,0,1,0,0




# Step 2. Compute Priors
<!-- 
$prior(w, i) = \frac{ \text{number of times the word} ~w~ \text{occurs at location} ~~ i}}{\text{sum of number of times any word occurs at location|~~ i}$ -->

<img width="30%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.5./priors.png">


In [3]:
df = df / 4
df

,0,1,2,3,4,5
!,0.0,0.00,0.00,0.00,0.00,0.25
.,0.0,0.00,0.25,0.00,0.00,0.00
a,0.0,0.00,0.25,0.00,0.00,0.00
be,0.5,0.50,0.00,0.25,0.25,0.25
dont,0.0,0.00,0.00,0.25,0.00,0.00
hard,0.0,0.00,0.00,0.00,0.00,0.25
is,0.0,0.00,0.00,0.00,0.25,0.00
kind,0.0,0.25,0.00,0.00,0.00,0.00
mean,0.0,0.00,0.00,0.00,0.00,0.25
not,0.0,0.00,0.00,0.25,0.00,0.00


# Step 3. Compute Conditional Probabilities 

Convert to Conditional Probabilities Matrix where rows are $word_t$ and columns are $word_{t+1}$

<img width="75%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.5./cps.png">

In [4]:
condP_df = pd.DataFrame(0, index = vocab, columns = vocab)
# the total number of times KIND appears in (t + 1) where BE is in (t), divided by the number of total times BE appears in the corpus
tmpdf = pd.DataFrame(0, index = vocab, columns = vocab)

def kthMatrix(k):
  kthList = [-1] * k

  for word in all_tokens:
    for prevWord in kthList:
      if prevWord == -1:
        continue
      tmpdf[word][prevWord] = tmpdf[word][prevWord] + 1

    kthList.pop(0)
    kthList.append(word)

kthMatrix(1)

for word in vocab:
  condP_df.loc[word] = tmpdf.loc[word] / ( df.loc[word].sum() * 4 )
condP_df

,to,not,or,is,be,dont,a,hard,there,square,kind,mean,.,student,!
to,0.000000,0.0,0.000000,0,1.0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0
not,1.000000,0.0,0.000000,0,0.0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0
or,0.000000,0.5,0.000000,0,0.5,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0
is,0.000000,0.0,0.000000,0,0.0,0,0.000000,1,0.000000,0.000000,0.000000,0.000000,0,0,0
be,0.142857,0.0,0.142857,0,0.0,0,0.142857,0,0.142857,0.142857,0.142857,0.142857,0,0,0
dont,0.000000,0.0,0.000000,0,1.0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0
a,0.000000,0.0,0.000000,0,0.0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,1,0
hard,0.000000,0.0,0.000000,0,1.0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0
there,0.000000,0.0,1.000000,0,0.0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0
square,0.000000,0.0,0.000000,0,0.0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,1


# Question 1. Prior Sampling

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.5./prior_sample.png">

In [5]:
from numpy.random import choice

def priorSample(bn):
  pickList = []
  condpList = []

  candidates, n_picks, P = ['to', 'be'], 1, [0.5, 0.5]
  biasedDraw = choice(candidates, n_picks, p = P)
  firstWord = biasedDraw.copy()
  sentence = biasedDraw[0] + ' '

    # what word can appear after the previous word (what word can be in t+1 based off of t)
    # put the cp's in the P list, put the words in candidates list
  for i in range(1, bn):
    P = []
    candidates = []
    currentWord = biasedDraw

    curDf = condP_df.loc[currentWord]
    curDf = (curDf != 0).all()

    for w in vocab:
      if curDf.loc[w] == True:
        candidates.append(w)

    for cand in candidates:
      condP = condP_df.loc[currentWord, cand]
      P.append(condP[0])

      # 'mean' has no t+1 value, so when currentWord == 'mean', candidate && P are [], choice then breaks
    if len(candidates) < 1:
      return sentence
    biasedDraw = choice(candidates, 1, p = P)
    sentence += biasedDraw[0] + ' '

  return sentence[:-1]

# Question 2. Rejection Sampling 

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.5./rejection_sampling.png">

In [6]:
from tqdm import tqdm

def rejectionSample(e, bn, n):

  sample = ""
  sentenceCount = 0
  validSentences = []
  for i in tqdm(range(0, n)):
    sample = priorSample(bn)
    if e in sample[:9]: # 0:9 because "to be or" is 8 chars
      sentenceCount += 1
      validSentences.append(sample)

  shakespeareSentenceCount = 0
  for sentence in validSentences:
    if sentence == "to be or not to be":
      shakespeareSentenceCount += 1
  
  return shakespeareSentenceCount / sentenceCount

rejectionSample("to be or", 6, 10000)

100%|██████████| 10000/10000 [02:21<00:00, 70.92it/s]


0.5258493353028065

<br/>
<hr/>
<br/>


<center>
<img width="60%" src="https://larspsyll.files.wordpress.com/2013/07/034_infinite_monkey_theorem3.gif?w=479&h=204&zoom=2">

</center>

<br/>
<hr/>
<br/>